<a href="https://colab.research.google.com/github/AlexandreSarmento/BooleanNetworkBioME/blob/master/K_means_binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BINARIZATION THROUGH CLUSTERING METHODS NOTEBOOK
### Here we are adapting a previously developed approach called iterative k-means binarization with a clustering depth (d) of 1 (KM1) or 2 (KM2) or 3 (KM3). The number of cluster (k) must be greater than the number of experimental measurement (n) k > n. It's well know that k = 2**d, therefore, here concerning to the experimental data provided by database we set d = 2, so k = 4. 

## Instaling Packages

In [ ]:
!pip3 install biopython

In [ ]:
!pip install bitarray

     |████████████████████████████████| 88 kB 4.2 MB/s 
  Created wheel for bitarray: filename=bitarray-2.3.4-cp37-cp37m-linux_x86_64.whl size=171949 sha256=661d0348f3fefeaeb0789e32835f599f77fc8ca9dbd6479ae3475e9d3a36365a
  Stored in directory: /root/.cache/pip/wheels/84/cc/5b/0e861bdb5294d22d2d4f595df936f964a95258387e11494d41
Successfully built bitarray


## Importing Modules

In [ ]:
import math, numpy, sys
from bitarray import bitarray
import csv, array
from Bio import Cluster # importar Biopython
import os
import shutil

## Setting variables to proxy time serie binarization

In [ ]:
number_of_folders = 25
counterF=1
while counterF <= number_of_folders:
    os.mkdir('proxy_results{}'.format(str(counterF)))
    counterF += 1 

In [ ]:
inputFileName = []  
outputFileName = []
KMX = 3
number_of_bin = 100*5
ID_of_group = list(range(1,number_of_folders+1))
for loop in ID_of_group:
  
  inputFileName.append('/content/grupo'+str(loop)+'_')
  outputFileName.append('/content/proxy_results'+str(loop)+'/proxy_grupo'+str(loop)+'_')

originalSeries = {}    # original series stored here as a list
binarySeries = {}    # structure same as the original but with binary series
inputoutput = {} 

# setting variables to experimental time serie binarization

In [ ]:
!mkdir breast_results
number_of_bin = 3
KMX = 2
inputFileName = '/content/breast_raw_data2_'
outputFileName = '/content/breast_results/breast_raw_data2_'
originalSeries = {}    # original series stored here as a list
binarySeries = {}    # structure same as the original but with binary series
inputoutput = {} # keys are combinations, with a list of compbinations that are outputs of it 

In [ ]:
def decrementalKMeanBinarization(orig, clusters, reduction=0):    
    clusterData = {}
    clusterRes = {}
    binSeries = {}
    if clusters < 2:
            return None
    while clusters >= 2:
            # 'NON' allows you to ignore specific species 
            for k,v in list(orig.items()):
                    if k == "NON":
                            continue
                    data = []
                    for i in range(len(v)):
                            row = [i,v[i]]
                            data.append(row)
                    x = numpy.vstack(v)
                    clusterData[k] = x
                                        # Bio.Cluster.kcluster()
                    idx, error, nfound = Cluster.kcluster(x, nclusters=clusters, method='a', dist='e')
                    clusterRes[k] = (idx, error, nfound)

            # replace the values with the averages of the clusters
            for k,r in list(clusterRes.items()):
                    colors = []
                    idx, error, nfound = r
                    y = clusterData[k]
                    x = numpy.arange(len(y))
                    sizes = {}
                    sums = {}
                    for i in range(clusters):
                            sizes[i] = 0
                            sums[i] = 0
                    for i in range(len(idx)):
                            sizes[idx[i]] += 1
                            sums[idx[i]] += y[i]
                    # culculate sums
                    for i in list(sums.keys()):
                            sums[i] /= sizes[i]
                            
                    # replace all the values with sums in the clusters
                    averaged = []
                    for i in idx:
                            averaged.append(sums[i][0])
                    orig[k] = averaged

            clusters //= 2
    # replace data with 0 and 1
    for k,v in list(orig.items()):
            if k == "NON":
                    continue
            mi = min(v)
            binVals = bitarray()
            for i in v:
                    if i == mi:
                            binVals.append(False)
                    else:
                            binVals.append(True)
            binSeries[k] = binVals
  
    return binSeries

# Main script to generate proxy time serie binarization

In [ ]:
for loopG in range(len(ID_of_group)):
    
    print "Group: " + str(loopG)
    
    for loopB in range(number_of_bin):
        
        print(str(loopB) + " of " + str(number_of_bin))
        bMethod = KMX 
        #Reduction methods
        reduction = 0
        print(inputFileName[loopG])
        csvfile = open(inputFileName[loopG] +'.csv', 'r') 
        r = csv.reader(csvfile)
        topLine = next(r)
        #topLine = r.next()
        order = []
        # grabs each species, 
        for t in topLine:
                # creates an empty list for each species
                originalSeries[t] = []
                order.append(t)
        for line in r:
                if line[0].startswith("#"):
                        continue
                words = line
                # fills the array for each species with the original data points
                for i in range(len(words)):
                        originalSeries[order[i]].append(float(words[i]))
        allConvergence = []
        allTimes = []
        # determines cluster depth (d). Number of clusters, k = 2^d 
        if bMethod > 0:
            clusters = int(math.pow(2,bMethod))
            binarySeries = decrementalKMeanBinarization(dict(originalSeries), clusters, reduction)
        for i in binarySeries:
            sampleSize2= len(binarySeries[i])
        # write binarized data to file
        f = open(outputFileName[loopG] + str(loopB) + '.csv', "wb")
        for item in binarySeries:
            f.write(bytes(item,'utf-8'))
            #f.write(str(item))
            values = str(binarySeries[item])
            values  = values.rstrip(')')
            values = values.rsplit('bitarray(')
            values = values[1]
            values = values.lstrip('\'')
            values = values.rstrip('\'')
            valueList = list(values)
            #tmp[1]
            for sample in valueList:
                f.write(bytes("," + sample,'utf-8'))
                #f.write("," + str(sample))
            #print(str(valueList)+',')
            f.write(bytes("\n",'utf-8'))
    
        # Close opened file
        f.close() 

# Main Script to generate experimental time serie binarization

In [ ]:
counter = 0
while (counter < number_of_bin):
    counter += 1
    print((str(counter) + " of " + str(number_of_bin) + "binarizations"))
    bMethod = KMX
    #Reduction methods
    reduction = 0
    csvfile = open(inputFileName +'.csv', 'r') 
    r = csv.reader(csvfile)
    
    topLine = next(r)
    order = []
    # grabs each species, 
    for t in topLine:
            # creates an empty list for each species
            originalSeries[t] = []
            order.append(t)
    for line in r:
            if line[0].startswith("#"):
                    continue
            words = line
            # fills the array for each species with the original data points
            for i in range(len(words)):
                    originalSeries[order[i]].append(float(words[i]))
    allConvergence = []
    allTimes = []
    # determines cluster depth (d). Number of clusters, k = 2^d 
    if bMethod > 0:
        clusters = int(math.pow(2,bMethod))
        binarySeries = decrementalKMeanBinarization(dict(originalSeries), clusters, reduction)
    for i in binarySeries:
        sampleSize2= len(binarySeries[i])
    # write binarized data to file
    f = open(outputFileName + str(counter) + '.csv', "wb")
    for item in binarySeries:
        f.write(bytes(item,'utf-8'))
        values = str(binarySeries[item])
        values  = values.rstrip(')')
        values = values.rsplit('bitarray(')
        values = values[1]
        values = values.lstrip('\'')
        values = values.rstrip('\'')
        valueList = list(values)
        for sample in valueList:
            f.write(bytes("," + sample,'utf-8'))
            #f.write("," + str(sample))
        f.write(bytes("\n",'utf-8'))

    f.close() 

## create or delete zip folders

In [ ]:
# here we are creating zip folders to each proxy group of input parameters binarization
for x in ID_of_group:
  shutil.make_archive('/content/proxy_results'+str(x), 'zip', '/content/proxy_results'+str(x))

In [ ]:
# here we are deleting zip folders to each proxy group of input parameters binarization
for j in ID_of_group:
  shutil.rmtree('/content/proxy_results'+str(j), ignore_errors=True)

In [ ]:
# here we are creating zip folders to breast time serie binarization
shutil.make_archive('/content/breast_results', 'zip', '/content/breast_results')

'/content/breast_results.zip'

In [ ]:
# here we are deleting zip folders to breast time serie binarization
shutil.rmtree('/content/glioma_results')

In [ ]:
!rm *.csv
!rm -rf *.zip

In [ ]:
%reset